## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-23-53-10 +0000,nypost,Real estate giant Cushman & Wakefield to hire ...,https://nypost.com/2025/11/11/us-news/real-est...
1,2025-11-11-23-48-58 +0000,nypost,Amazon cuts 700 jobs in NYC alone on quest to ...,https://nypost.com/2025/11/11/business/amazons...
2,2025-11-11-23-46-47 +0000,nyt,Justice Department to Investigate Protests at ...,https://www.nytimes.com/2025/11/11/us/justice-...
3,2025-11-11-23-43-31 +0000,nyt,Key Netanyahu Minister Steps Down From Israel’...,https://www.nytimes.com/2025/11/11/world/middl...
4,2025-11-11-23-42-39 +0000,nyt,Senator Criticizes Rubio for Paying $7.5 Milli...,https://www.nytimes.com/2025/11/11/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
65,trump,38
50,shutdown,29
37,government,22
367,new,16
101,end,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
306,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,140
43,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,113
282,2025-11-11-03-12-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,107
155,2025-11-11-17-01-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,102
10,2025-11-11-23-36-03 +0000,nypost,Supreme Court extends order blocking full SNAP...,https://nypost.com/2025/11/11/us-news/supreme-...,97


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
306,140,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
43,62,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
106,48,2025-11-11-19-22-21 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
261,45,2025-11-11-08-43-05 +0000,wsj,"Stock Market Today: Dow Jumps, Nasdaq Slips; A...",https://www.wsj.com/livecoverage/stock-market-...
71,43,2025-11-11-20-58-15 +0000,nypost,Anti-parade Zohran Mamdani touts affordable ho...,https://nypost.com/2025/11/11/us-news/anti-par...
42,37,2025-11-11-22-23-52 +0000,nypost,Ex-boyfriend busted for stabbing Harlem woman ...,https://nypost.com/2025/11/11/us-news/ex-boyfr...
10,32,2025-11-11-23-36-03 +0000,nypost,Supreme Court extends order blocking full SNAP...,https://nypost.com/2025/11/11/us-news/supreme-...
33,31,2025-11-11-22-36-55 +0000,nypost,Florida parents arrested after 4-year-old twin...,https://nypost.com/2025/11/11/us-news/florida-...
107,30,2025-11-11-19-03-05 +0000,nypost,China plans to block rare earth shipments to U...,https://nypost.com/2025/11/11/business/china-p...
171,29,2025-11-11-16-24-12 +0000,bbc,Israel passes first reading of bill proposing ...,https://www.bbc.com/news/articles/cpv13rxpgk9o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
